In [28]:
import faiss
import numpy as np
import time

In [32]:
def load_data(filename):
    with open(filename, "rb") as file:
        dim_bytes = file.read(4)  # Read 4 bytes for dimension
        dim = int.from_bytes(dim_bytes, byteorder='little')  # Convert bytes to integer for dimension

        file.seek(0, 2)  # Move the file pointer to the end
        fsize = file.tell()  # Get the file size
        num = fsize // ((dim + 1) * 4)  # Calculate the number of data points

        file.seek(0)  # Move the file pointer back to the beginning
        data = np.empty((num, dim), dtype=np.float32)  # Create an empty numpy array to store data

        for i in range(num):
            file.seek(4, 1)  # Move the file pointer forward by 4 bytes to skip index
            data[i] = np.fromfile(file, dtype=np.float32, count=dim)  # Read dim number of float values

    return data, num, dim

def ivecs_read(fname):
    a = np.fromfile(fname, dtype='int32')
    d = a[0]
    # Wenqi: Format of ground truth (for 10000 query vectors):
    #   1000(topK), [1000 ids]
    #   1000(topK), [1000 ids]
    #        ...     ...
    #   1000(topK), [1000 ids]
    # 10000 rows in total, 10000 * 1001 elements, 10000 * 1001 * 4 bytes
    return a.reshape(-1, d + 1)[:, 1:].copy()

# Change to the path of your data
data_load, points_num, dim = load_data("./sift/sift_base.fvecs")
query_load, query_num, query_dim = load_data("./sift/sift_query.fvecs")
assert(dim == query_dim)

gt_load = ivecs_read("./sift/sift_groundtruth.ivecs")

print("data_load: ", np.shape(data_load))
print("query_load: ", np.shape(query_load))
print("gt_load: ", np.shape(gt_load))

data_load:  (1000000, 128)
query_load:  (10000, 128)
gt_load:  (10000, 100)


In [34]:
# res = faiss.StandardGpuResources()  # use a single GPU

cpu_index = faiss.IndexFlatL2(dim)
# make it into a gpu index
gpu_index = faiss.index_cpu_to_all_gpus(cpu_index)

In [35]:
gpu_index.add(data_load)         # add vectors to the index
print(gpu_index.ntotal)

k = 10                          # we want to see 4 nearest neighbors
total_time = 0.0
for i in range(query_num):
    start_time = time.perf_counter()
    D, I = gpu_index.search(query_load[i:i+1], k)  # actual search
    end_time = time.perf_counter()
    total_time = total_time + end_time - start_time

print(total_time)

1000000
33.82398530095816


In [ ]:
1e3 * total_time / query_num

3.3823985300958155